# NML vs. quantile histograms

In [1]:
import capnp
import codecs
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import statistics

capnp.remove_import_hook()
interface_capnp = capnp.load("../../../cpp-tool/capnproto/interface.capnp", imports=["/usr/local/include"])

import sys
sys.path.append("../../..")
from experiments.evaluation import Evaluator
from experiments.modelgeneration import ModelGenerator

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /root/.vscode-server/bin/7f329fe6c66b0f86ae1574c2911b681ad5a45d63


In [2]:
client = capnp.TwoPartyClient("cpp-tool:21654")
processDataUpdates = client.bootstrap().cast_as(interface_capnp.ProcessDataUpdates)

In [3]:
# initialize objects
evaluator = Evaluator(interface_capnp, processDataUpdates)
modelGen = ModelGenerator(interface_capnp, processDataUpdates, processDataUpdates)

## Artificial data

In [4]:
!capnp convert json:binary ../../../cpp-tool/capnproto/interface.capnp RuleModel < ../data-generation/dataGenerationModel.json > models/--0--true-model.bin
with open("models/--0--true-model.bin", "rb") as file:
    model = interface_capnp.RuleModel.read(file)
print(processDataUpdates.modelToString(model).wait().string)

IF light1 = red THEN light2 IN {red}
IF light2 = green THEN cost IN [0.000000, 10.000000]
IF light2 = yellow THEN DELTA cost IN [12.000000, 15.000000]
IF light1 = green THEN agility = 42.000000
IF light1: red -> yellow THEN agility = agility_prev * 0.900000
IF light2: yellow -> red THEN cost = cost_prev - 7.000000



### Single example

In [5]:
eventLogPromise = processDataUpdates.generateEventLog(model=model, numberOfTraces=100, traceLength=20)
resolvedEventLogPromise = eventLogPromise.wait()

In [6]:
%%time
discretizationMode = interface_capnp.DiscretizationMode.new_message()
discretizationMode.nml = None
modelPromise = processDataUpdates.findModel(resolvedEventLogPromise.log, discretizationMode=discretizationMode)
foundModel = modelPromise.wait().model
print(processDataUpdates.modelToString(foundModel).wait().string)

IF light1: red -> yellow THEN agility = agility_prev * 0.900104
IF light2 = yellow THEN DELTA cost IN [12.000000, 15.000000]
IF light1 = red THEN light2 IN {red}
IF light1 = green THEN agility = 42.000000
IF light2: yellow -> red THEN cost = cost_prev - 7.000000
IF NOT (light1 = green) THEN agility IN [0.200000, 499.800000]
IF light2 = green THEN cost IN [0.000000, 10.000000]

CPU times: user 1.59 ms, sys: 1.93 ms, total: 3.51 ms
Wall time: 5min 48s


In [7]:
%%time
discretizationMode = interface_capnp.DiscretizationMode.new_message()
discretizationMode.quantiles = None
modelPromise = processDataUpdates.findModel(resolvedEventLogPromise.log, discretizationMode=discretizationMode)
foundModel = modelPromise.wait().model
print(processDataUpdates.modelToString(foundModel).wait().string)

IF light1 = green THEN agility = 42.000000
IF light2 = yellow THEN DELTA cost IN [12.000000, 15.000000]
IF light1 = red THEN light2 IN {red}
IF light1: red -> yellow THEN agility = agility_prev * 0.900104
IF light2: yellow -> red THEN cost = cost_prev - 7.000000
IF NOT (light1 = green) THEN agility IN [0.200000, 499.800000]
IF light2 = green THEN cost IN [0.000000, 10.000000]

CPU times: user 2.62 ms, sys: 0 ns, total: 2.62 ms
Wall time: 20.4 s


### Systematic comparison

In [5]:
experimentName = "models"
numberOfDatasets = 10
logLength = 2000
traceLength = 10

In [6]:
emptyModel = interface_capnp.RuleModel(rules=[], variables=list(model.variables))
modelGen.saveModel(experimentName, "", "", 0, "", "empty", emptyModel)

In [7]:
numberOfTraces = logLength // traceLength
for j in range(numberOfDatasets): # iterate over the event logs per generating model
    # generate event log
    log = processDataUpdates.generateEventLog(model, numberOfTraces, traceLength).wait().log

    # estimate models from event log
    myModel = processDataUpdates.findModel(log).wait().model.as_builder()
    discretizationMode = interface_capnp.DiscretizationMode.new_message()
    discretizationMode.nml = None
    baselineModel = processDataUpdates.findModel(log, discretizationMode=discretizationMode).wait().model.as_builder()

    # save models for later evaluation
    modelGen.saveModel(experimentName, 0.0, logLength, 0, j, "quantiles", myModel)
    modelGen.saveModel(experimentName, 0.0, logLength, 0, j, "nml", baselineModel)

## Evaluation

In [8]:
table = evaluator.createExperimentTable("models",
    r"(\d\.\d+)?-(\d+)?-(\d+)-(\d+)?-(.+)-model\.bin",
    ["noiseProportion", "logLength", "modelID", "logVariant", "discretizationMode"])

In [9]:
metrics = ["strictF1", "looseF1"]
evaluator.addMetricsToExperimentTable("models", table, metrics)

Computing looseF1: 100%|██████████| 22/22 [00:00<00:00, 129.19it/s]


In [10]:
aggregationColumns = ["logLength", "noiseProportion", "discretizationMode"]
tableAggregated = evaluator.aggregateMetrics(table, aggregationColumns, metrics)

In [12]:
tableAggregated

strictF1                     \
                                                lowIqr   midIqr   highIqr   
logLength noiseProportion discretizationMode                                
2000      0.0             nml                 0.857143  0.89011  0.923077   
                          quantiles           0.857143  0.89011  0.923077   
NaN       NaN             empty               0.000000  0.00000  0.000000   
                          true                1.000000  1.00000  1.000000   

                                               looseF1                     
                                                lowIqr   midIqr   highIqr  
logLength noiseProportion discretizationMode                               
2000      0.0             nml                 0.857143  0.89011  0.923077  
                          quantiles           0.857143  0.89011  0.923077  
NaN       NaN             empty               0.000000  0.00000  0.000000  
                          true                1.000000  1.00000  1.000000

In [13]:
runtimes = pd.read_csv("runtimes.csv")
runtimesAggregated = evaluator.aggregateMetrics(runtimes, ["Histogram estimation method"], ["Runtime [sec]"])

In [14]:
runtimesAggregated

Runtime [sec]                        
                                   lowIqr      midIqr     highIqr
Histogram estimation method                                      
MDL                            322.269500  328.516500  334.339750
quantiles                        0.000205    0.000213    0.000215

## Real-world data (Sepsis)

In [3]:
import pm4py
log = pm4py.read_xes("datasets/Sepsis Cases - Event Log.xes")
log.sort_values("time:timestamp", axis="rows", inplace=True)
log.drop(["lifecycle:transition", "time:timestamp"], axis="columns", inplace=True)

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|████████████████████████████████████████████| 1050/1050 [00:00<00:00, 2345.99it/s]


In [4]:
# only keep numerical attributes, case ID and activity name
# categorical attributes are unaffected by histogram estimation
columnsToDrop = [col for col in log.columns if "concept:name" not in col and log[col].dtype != np.float64]
log.drop(columns=columnsToDrop, inplace=True)

In [5]:
from dataframetocapnp import dataframeToMessage

eventLogMessage = dataframeToMessage(log, fillMissingValues=True)

Constructing traces: 100%|█████████████████████████████████████████████████████████| 1050/1050 [00:02<00:00, 415.02it/s]


### Model search comparison

In [6]:
%%time
discretizationMode = interface_capnp.DiscretizationMode.new_message()
discretizationMode.nml = None
modelPromise = processDataUpdates.findModel(eventLogMessage, discretizationMode=discretizationMode)
foundModel = modelPromise.wait().model
print(processDataUpdates.modelToString(foundModel).wait().string)

IF TRUE THEN Age = Age_prev + 0.000000
IF NOT (concept:name = CRP) THEN CRP = CRP_prev + 0.000000
IF NOT (concept:name = LacticAcid) THEN LacticAcid = LacticAcid_prev + 0.000000
IF NOT (concept:name = Leucocytes) THEN Leucocytes = Leucocytes_prev + 0.000000

CPU times: user 26.3 ms, sys: 17.3 ms, total: 43.6 ms
Wall time: 11min 34s


In [7]:
%%time
discretizationMode = interface_capnp.DiscretizationMode.new_message()
discretizationMode.quantiles = None
modelPromise = processDataUpdates.findModel(eventLogMessage, discretizationMode=discretizationMode)
foundModel = modelPromise.wait().model
print(processDataUpdates.modelToString(foundModel).wait().string)

IF TRUE THEN Age = Age_prev + 0.000000
IF NOT (concept:name = CRP) THEN CRP = CRP_prev + 0.000000
IF NOT (concept:name = LacticAcid) THEN LacticAcid = LacticAcid_prev + 0.000000
IF NOT (concept:name = Leucocytes) THEN Leucocytes = Leucocytes_prev + 0.000000

CPU times: user 16.7 ms, sys: 16.5 ms, total: 33.1 ms
Wall time: 8min 51s


Again, the same rules are found regardless of the discretization method. :)

## Why do we find global ranges?

In [9]:
!capnp convert json:binary ../cpp-tool/capnproto/interface.capnp RuleModel < histogramTestModel.json > /tmp/histogramTestModel.bin
with open("/tmp/histogramTestModel.bin", "rb") as file:
    model = interface_capnp.RuleModel.read(file)
print(processDataUpdates.modelToString(model).wait().string)

IF TRUE THEN CRP = CRP_prev + 0.000000
IF concept:name = CRP THEN CRP IN [5.000000, 573.000000]



In [10]:
processDataUpdates.calculateMdlScore(log=eventLogMessage, model=model).wait().result

272196.0726131758

In [8]:
log[log["case:concept:name"] == "A"]

concept:name   Age case:concept:name  Leucocytes    CRP  LacticAcid
0    ER Registration  85.0                 A         NaN    NaN         NaN
1         Leucocytes  85.0                 A         9.6    NaN         NaN
2                CRP  85.0                 A         9.6   21.0         NaN
3         LacticAcid  85.0                 A         9.6   21.0         2.2
4          ER Triage  85.0                 A         9.6   21.0         2.2
5   ER Sepsis Triage  85.0                 A         9.6   21.0         2.2
6          IV Liquid  85.0                 A         9.6   21.0         2.2
7     IV Antibiotics  85.0                 A         9.6   21.0         2.2
8       Admission NC  85.0                 A         9.6   21.0         2.2
10        Leucocytes  85.0                 A         8.7   21.0         2.2
9                CRP  85.0                 A         8.7  109.0         2.2
12               CRP  85.0                 A         8.7   47.0         2.2
11        Leucocytes  85.0                 A         9.6   47.0         2.2
14               CRP  85.0                 A         9.6   15.0         2.2
13        Leucocytes  85.0                 A        10.7   15.0         2.2
16        Leucocytes  85.0                 A        13.0   15.0         2.2
15               CRP  85.0                 A        13.0    9.0         2.2
17        Leucocytes  85.0                 A        11.3    9.0         2.2
18               CRP  85.0                 A        11.3    9.0         2.2
19               CRP  85.0                 A        11.3    6.0         2.2
20        Leucocytes  85.0                 A        10.9    6.0         2.2
21         Release A  85.0                 A        10.9    6.0         2.2

Global range rules are found to fix prediction errors of the always true update rule that predicts the same value.

### Artificial example

In [12]:
import random
caseIds = [str(j) for i in range(1000) for j in [i] * 10]
catVariable = random.choices(["red", "green"], [1, 4], k=10*1000)
numVariable = [float("nan")] * (10*1000)
for i in range(0, 10000, 10):
    numValueForTrace = round(random.uniform(0, 100), 1)
    setFirstValue = False
    for j in range(10):
        if catVariable[i+j] == "red":
            setFirstValue = True
        if setFirstValue:
            numVariable[i+j] = numValueForTrace

data = pd.DataFrame({"case:concept:name": caseIds, "cat": catVariable, "num": numVariable})
data.head()

case:concept:name    cat   num
0                 0  green   NaN
1                 0  green   NaN
2                 0  green   NaN
3                 0    red  25.7
4                 0    red  25.7